In [88]:
import datasets
import utils
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
import pandas as pd
import pickle_utils as pu

In [102]:
#dset = datasets.datasets()["BostonHousing"][0].astype(np.float64)
dset = pu.load("impute_benchmark/amputed_BostonHousing_MCAR_rows_0.3.pkl.gz")
test_mask = np.random.rand(len(dset)) < 0.2

norm_mean = dset[~test_mask].mean()
norm_std = dset[~test_mask].std()

dset = (dset - norm_mean) / norm_std
dset = dset.fillna(0)

y_train = dset.values[~test_mask, 0]
X_train = dset.values[~test_mask, 1:]
y_test = dset.values[test_mask, 0]
X_test = dset.values[test_mask, 1:]

dset

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
1,-0.439541,0.232202,-1.318699,0.675028,-0.124403,0.498903,-0.072483,0.128169,-0.979145,-0.672666,-1.536773,0.440875,-1.087087
2,0.000000,0.000000,-0.610376,0.675028,0.000000,0.254145,0.409962,0.537514,-0.863680,-0.996071,-0.352908,0.440875,-0.507591
3,-0.437256,0.000000,-0.610376,-1.477902,0.000000,0.000000,-0.216865,0.537514,0.000000,0.000000,-0.352908,0.398234,0.000000
4,-0.436702,-0.519476,0.000000,-1.477902,0.000000,0.000000,-0.755654,0.000000,0.000000,0.000000,0.000000,0.417093,-1.371263
5,0.000000,0.000000,0.000000,0.675028,-0.786392,1.408002,-0.459848,0.000000,-0.748215,-1.115851,0.073283,0.440875,0.000000
6,-0.436977,0.000000,-1.338044,0.675028,0.000000,0.000000,0.000000,1.048600,0.000000,0.000000,0.000000,0.411749,0.000000
7,-0.430607,0.000000,0.000000,-1.477902,0.000000,-0.395892,0.000000,0.814455,0.000000,-0.582831,0.000000,0.427255,-0.049288
8,-0.424476,0.002522,-0.491330,0.675028,-0.240251,-0.141599,1.015660,0.996469,-0.517284,-0.582831,-1.584128,0.440875,0.886821
9,0.000000,0.000000,0.000000,-1.477902,-0.240251,0.000000,0.000000,0.000000,-0.517284,0.000000,0.000000,0.333277,0.000000
10,0.000000,0.002522,0.000000,-1.477902,-0.240251,0.000000,0.000000,0.000000,0.000000,0.000000,-1.584128,0.000000,0.000000


In [103]:
np.all(np.isfinite(y_train)), np.all(np.isfinite(X_train)), np.all(np.isfinite(y_test)), np.all(np.isfinite(X_test))

(True, True, True, True)

In [104]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1,
                           max_features=int(np.floor(X_train.shape[1]**.5)),
                           bootstrap=False,
                           min_samples_split=5)
rf.fit(X_train, y_train)
train_perf = rf.score(X_train, y_train)
test_perf = rf.score(X_test, y_test)
print("Test: {:.4f}, Training: {:.4f}".format(test_perf, train_perf))
y, _ = m.predict(X_train)
print("Train:", rmse(y_train, y.flatten()))
y, _ = m.predict(X_test)
print("Test:", rmse(y_test, y.flatten()))

Test: 0.2045, Training: 0.9087
Train: 0.671255061871
Test: 0.980223486398


In [105]:
def r_squared(y_true, y):
    u = np.sum((y-y_true)**2)
    v = np.sum((y_true - np.mean(y_true))**2)
    return 1 - u/v
def rmse(y_true, y):
    return np.mean((y_true - y)**2)

In [110]:
import GPy
m = GPy.models.SparseGPRegression(X_train, y_train[:,np.newaxis],
                            kernel=GPy.kern.Matern52(X_train.shape[1], ARD=False) + GPy.kern.White(X_train.shape[1]))
m.optimize()
y, _ = m.predict(X_train)
print("Train:", rmse(y_train, y.flatten()))
y, _ = m.predict(X_test)
print("Test:", rmse(y_test, y.flatten()))

Train: 0.488219156171
Test: 0.930104888792


In [59]:
dset.keys()

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'b', 'lstat'],
      dtype='object')

In [39]:
import selu
import denoising_ae as dae
import imp
imp.reload(dae)
imp.reload(selu)

tf_float = tf.float32
tf.reset_default_graph()

nn_X = tf.placeholder(tf_float, shape=[None, X_train.shape[1]], name="X")
nn_y = tf.placeholder(tf_float, shape=[None, 1], name="y")
nn_kp = tf.placeholder(tf_float, shape=[], name="keep_prob")
nn_lr = tf.placeholder(tf_float, shape=[], name="learning_rate")


nn_preds = dae.FC_net(nn_X, [(1024, selu.nlin)]*8 + [(1, None)], selu.initializer, keep_prob=nn_kp)
nn_loss = tf.reduce_mean(tf.squared_difference(nn_preds, nn_y))
nn_train = tf.train.GradientDescentOptimizer(nn_lr).minimize(nn_loss)

In [40]:
from tqdm import tqdm
import itertools

sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

batch_size = 128

num_batches = int(np.ceil(X_train.shape[0] / batch_size))

pbar = tqdm(itertools.count(0))
for step in pbar:
    i = step % num_batches
    # Populate feed_dict; duplicated code for num and cat
    s = slice(batch_size * i, batch_size * (i + 1))
    feed_dict = {nn_kp: 0.95,
                 nn_lr: 0.001 if step < 800 else 0.0001,
                 nn_X: X_train[s],
                 nn_y: y_train[s, np.newaxis],
                }
    del i
    
    if step % 100 == 0:
        _, l, y = sess.run([nn_train, nn_loss, nn_preds], feed_dict)
        d = "Loss: {} ".format(l)
        d += "Train: {} ".format(r_squared(y_train[s], y.ravel()))
        y = sess.run(nn_preds, {nn_kp: 1.0,
                                nn_X: X_test,
                                nn_y: y_test[:, np.newaxis]})
        d += "Test: {} ".format(r_squared(y_test, y.ravel()))
        pbar.set_description(d)
    else:
        sess.run(nn_train, feed_dict)


0it [00:00, ?it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 1it [00:00,  3.25it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 3it [00:00,  4.23it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 5it [00:00,  5.52it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 8it [00:00,  7.19it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 11it [00:00,  9.28it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 14it [00:00, 11.47it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 17it [00:01, 13.87it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 20it [00:01, 16.15it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 : 23it [00:01, 18.33it/s]
Loss: 2.2472610473632812 Train: -1.3396346953249103 Test: -3.2978709281776 

KeyboardInterrupt: 

In [41]:
tf.summary.FileWriter("faceoff/", graph=sess.graph)

Loss: 0.055597126483917236 Train: 0.9541863795146673 Test: 0.9235167656130201 : 3687it [02:48, 21.89it/s]

In [23]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process.kernels import WhiteKernel, RBF
import time

# Fit KernelRidge with parameter selection based on 5-fold cross validation
param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "kernel": [RBF(l)
                         for l in np.logspace(-2, 2, 10)]}
kr = GridSearchCV(KernelRidge(), cv=5, param_grid=param_grid)
stime = time.time()
kr.fit(X_train[:100], y_train[:100])
print("Time for KRR fitting: %.3f" % (time.time() - stime))

Time for KRR fitting: 0.549


In [18]:
Z = np.random.rand(1000, X_train.shape[1])
rbf = GPflow.kernels.RBF(X_train.shape[1], variance=1, ARD=True)
m = GPflow.sgpr.SGPR(X_train, y_train[:,None], kern=rbf, Z=Z)
def logger(x):
    if (logger.i % 10) == 0:
        print(x, m._objective(x)[0])
    logger.i+=1
logger.i = 0

m.optimize(method=tf.train.AdamOptimizer(), callback=logger)



[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 181586.72233875]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 180537.74127184]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 179500.65675795]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 178476.56694212]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 177466.08404523]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 176469.42592986]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 175486.5617425]
[ 0.42271147  0.10330646  0.66634543 ...,  0.54132327  0.54132327
  0.54132327] [ 174517.32678109]
Caught KeyboardInterrupt, setting model                  with most recent state.


In [24]:
y, _ = m.predict_y(X_train)
print("Train:", r_squared(y_train, y.flatten()))
y, _ = m.predict_y(X_test)
print("Test:", r_squared(y_test, y.flatten()))

Train: -4.42959616572
Test: -4.39652614283
